# Introduction to NLP Fundatmentals in TensorFlow

NLP has the goal of deriving information out of natural language

Another common term for NLP problem is seq2seq

## Check for GPU

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


## Downloading helper function inside the folder


! wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [2]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we're going ti be using is Kaggle's introduction to NLP dataset. A classification problem

[Competition Link](https://www.kaggle.com/competitions/nlp-getting-started/overview)

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data('nlp_getting_started.zip')

--2024-03-12 11:10:55--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2404:6800:4009:805::201b, 2404:6800:4009:806::201b, 2404:6800:4009:809::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2404:6800:4009:805::201b|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: 'nlp_getting_started.zip.1'

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.1s    

2024-03-12 11:10:55 (4.80 MB/s) - 'nlp_getting_started.zip.1' saved [607343/607343]



## Become one with the data

In [4]:
# Read thed data
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac= 1, random_state= 42)

train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class
train_df['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [8]:
# How may samples
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualise some random training examples
import random
random_index = random.randint(0, len(train_df) - 5)

for row in train_df_shuffled[['text', 'target']][random_index: random_index + 5].itertuples():
    _, text, target = row
    print(f"Target: {'Disaster' if target == 1 else 'Not a disaster'}")
    print(f"Text: {text}")
    print("----\n")


Target: Not a disaster
Text: Mike Magner Discusses A Trust Betrayed: http://t.co/GETBjip5Rh via @YouTube #military #veterans #environment
----

Target: Not a disaster
Text: Beware of your temper and a loose tongue! These two dangerous weapons combined can lead a person to the Hellfire #islam!
----

Target: Disaster
Text: Flood: Two people dead 60 houses destroyed in Kaduna: Two people have been reportedly killed and 60 houses ut... http://t.co/BDsgF1CfaX
----

Target: Disaster
Text: MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ... - ABC Onlin... http://t.co/N3lNdJKYo3 G #Malaysia #News
----

Target: Not a disaster
Text: Whenever I have a meltdown and need someone @Becca_Caitlyn99 is always like 'leaving in 5' and I don't know how I got so lucky #blessed
----



### Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

# Use train_test_split() to split the trianing data into train and validation dataset

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(), test_size= 0.1, random_state= 42)

In [11]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [12]:
# Check the first 10 sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to nuumbers.

There are a few ways to do this:
* Tokenization
* Embedding

### Text vectorization (tokenization)

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use defalut TextVectorization parameter
text_vectorizer = TextVectorization(max_tokens= None, # how many words in vocab
                                    standardize= 'lower_and_strip_punctuation',
                                    split= 'whitespace',
                                    ngrams= None, # creates a group of words
                                    output_mode= 'int', # in which format the output should be
                                    output_sequence_length= None, # how long deos the output sequence should be of
                                    )

In [14]:
# Find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences]) // len(train_sentences))

14

In [15]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocab
max_length = 15 # max length our sequences will be

text_vectorizer = TextVectorization(max_tokens= max_vocab_length,
                                    output_mode= 'int',
                                    output_sequence_length= max_length)

In [16]:
# Fit the text vectorization to training set
text_vectorizer.adapt(train_sentences)

In [17]:
# Create a sample sentence and tokenize it
sample_sentences = "There's a flood in my street!"

text_vectorizer([sample_sentences])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Choose a random sentences from the training sentences
random_sentences = random.choice(train_sentences)
print(f"Original text: {random_sentences}")
print(f"Vectorized sentence: {text_vectorizer([random_sentences])}")

Original text: -??-
; kitana
? her fatalities slay me
ÛÓkody? (Vine by @KOMBATFANS33) https://t.co/uMajwSNLUF
Vectorized sentence: [[   1   81  301 8417   31 6431 1927   18    1    1    0    0    0    0
     0]]


In [21]:
words_in_vocab = text_vectorizer.get_vocabulary() # retrieves all of the unique words

tope_5_words = words_in_vocab[:5] # gets the most common words
bottom_5_words = words_in_vocab[-5:] # gets the least common words

print(f"Number fo words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {tope_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number fo words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding layer

We're going to use Tensorflow Embedding layer.


The parameters we care most about our embedding layers:
* `input_dim`
* `output_dim`
* `input_length`

In [23]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim= max_vocab_length, # set input shape
                             output_dim= 128,
                             input_length= max_length # how long is each input
                             )

embedding

In [36]:
# Get a ranomd sentence from the training set
random_sentences = random.choice(train_sentences)

print(f"Original sentences: {random_sentences}")

# Embed the random sentences
sample_embde = embedding(text_vectorizer([random_sentences]))
sample_embde

Original sentences: that horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.043427  , -0.02271678, -0.00248923, ..., -0.03483094,
         -0.01346254, -0.03058391],
        [ 0.03267677,  0.04190341, -0.04285845, ...,  0.0135429 ,
         -0.02445282, -0.03918562],
        [ 0.01630927, -0.03321409, -0.00125452, ...,  0.02345994,
          0.02224901,  0.00741848],
        ...,
        [-0.02616034,  0.00962585,  0.0474998 , ...,  0.03219023,
          0.0456875 ,  0.04080323],
        [-0.01575265,  0.04413174,  0.01501777, ..., -0.04161366,
         -0.03297486,  0.00340591],
        [-0.01612275,  0.04820825,  0.0210881 , ...,  0.02680815,
          0.03131999, -0.00614514]]], dtype=float32)>

In [37]:
# Check out a single token's embedding
sample_embde[0][0], sample_embde[0][0].shape, random_sentences[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.043427  , -0.02271678, -0.00248923, -0.04603863,  0.00509753,
         0.01818783, -0.00246652,  0.01118038,  0.00157026,  0.04659058,
        -0.01123718, -0.00972148,  0.04788823,  0.01407531, -0.02020611,
        -0.03455592,  0.04628216, -0.04112779,  0.04135301, -0.02473733,
         0.03040474,  0.02041018,  0.00029361, -0.04249674,  0.0377947 ,
        -0.00522221,  0.03419198, -0.01971959,  0.02616808,  0.03351443,
        -0.0458042 , -0.02779064,  0.04195997, -0.02212015, -0.03683157,
         0.00789969,  0.02253792, -0.04799919, -0.03994947, -0.04821298,
        -0.04259375,  0.02873099,  0.03656069, -0.04618149, -0.04235861,
        -0.03062811,  0.02972574, -0.01694834,  0.03712305,  0.03289719,
         0.04936178, -0.01675064, -0.04377675,  0.02584152,  0.04081562,
        -0.00031313, -0.00864778, -0.00172927,  0.02708354, -0.0185649 ,
        -0.04001318,  0.00172317,  0.01937211,  0.00556774,  0.00397693,
  

## Modeling a text dataset

Experiments:

* Model 0: Naive Bayes
* Model 1: Feed-forward neural network
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: Tensorflow Hub pretrained fetaure extractor
* Model 7: Same as Model 6 but with 10% of the data

### Model 0: Getting a baseline

> **Note**: It is common practice to use non-DL algorithm as a baseline because of their speed and then alter using DL to see if you can improve upon them

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenizer and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert the text data into numerical form using tf-idf technique
    ("clf", MultinomialNB()) # classifier for the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [40]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves: {baseline_score * 100 :.2f}%")

Our baseline model achieves: 79.27%


In [42]:
# Make prediction
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [52]:
# Create an evaluation function to evaluate the model
# A function which returns a dictionary of accuracy, precision, recall and f1-score
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def evaluation(y_true, y_pred):
    '''
    This function gives the evaluation of the classification model in a dictionary form (Accuracy, Precision, Recall and F1-score)

    Parameters:
        y_true: The true labels of the evaluation data
        y_pred: The predicted labels of the evaluation data

    Returns:
        A dictionary with keys: `accuracy`, `precision`, `recall`, `f1-score`
    '''

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1score = f1_score(y_true, y_pred, average= 'weighted') # we are using weighted  average because the dataset is slightly imbalanced

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1-score': f1score
    }

In [84]:
baseline_results = evaluation(y_true= val_labels, 
           y_pred= baseline_preds)

### Model 1: A Simple dense model

In [54]:
# Create directory to save tensorboard callback logs
SAVED_DIR = "model_logs"

In [73]:
# Build model with the Functiobnal API

inputs = layers.Input(shape= (1, ), dtype= tf.string)
x = text_vectorizer(inputs) # convert text to numbers
x = embedding(x) # convert numbers to matrix
x = layers.GlobalAveragePooling1D(name= 'global_avg_pool_layer')(x)
outputs = layers.Dense(1, activation= 'sigmoid', name= 'output_layer')(x)

model_1 = tf.keras.Model(inputs, outputs, name= 'model_1_dense')

In [74]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glo  (None, 128)               0         
 balAveragePooling1D)                                            
                                                                 
 output_layer (Dense)        (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [75]:
# Compile the model
model_1.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.legacy.Adam(),
                metrics= ['accuracy'])

In [76]:
model_1_history = model_1.fit(x = train_sentences,
                              y= train_labels,
                              epochs= 5,
                              validation_data= (val_sentences, val_labels),
                              callbacks= [create_tensorboard_callback(SAVED_DIR, 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20240312-130658
Epoch 1/5
215/215 [==============================] - 5s 16ms/step - loss: 0.6111 - accuracy: 0.6871 - val_loss: 0.5362 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4403 - accuracy: 0.8183 - val_loss: 0.4701 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3456 - accuracy: 0.8610 - val_loss: 0.4577 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2838 - accuracy: 0.8920 - val_loss: 0.4646 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2376 - accuracy: 0.9099 - val_loss: 0.4869 - val_accuracy: 0.7887


In [77]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.4869 - accuracy: 0.7887


[0.4868801534175873, 0.7887139320373535]

In [79]:
model_1_preds = model_1.predict(val_sentences)
model_1_preds.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [83]:
model_1_results = evaluation(y_true= val_labels,
           y_pred= tf.round(tf.squeeze(model_1_preds)))

In [85]:
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False,  True, False])

## Visualizing learned embeddings

In [87]:
# Get the vocabulary from the text vectorization
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [88]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glo  (None, 128)               0         
 balAveragePooling1D)                                            
                                                                 
 output_layer (Dense)        (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

Now we've got the embedding matrix, let's see how to visualize it. To do so, Tensorflow has a handy tool called [projector](https://projector.tensorflow.org).

In [94]:
# Get the weight matrix of the embedding layer
embed_weights = model_1.get_layer('embedding').get_weights()
embed_weights[0].shape

(10000, 128)

In [96]:
# Create embedding files
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[0][index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

### Model 2: LSTM

LSTM = Long Short Term Memory

Architecture of the model:

```
Input(text) -> Tokenize -> Embedding -> Layers -> Output

```

In [126]:
# Create an LSTM Model
inputs = layers.Input(shape = (1, ), dtype= "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(units= 64, return_sequences= True, name= 'LSTM_layer_1')(x) # when stacking RNN cells together you need to set return_sequences = True 
# x = layers.LSTM(64, name= 'LSTM_layer_2')(x)
x = layers.GlobalAveragePooling1D(name= 'global_avg_pool_layer')(x)
x = layers.Dense(64, activation= 'relu', name= 'dense_layer')(x)
outputs = layers.Dense(1, activation= 'sigmoid', name= 'output_layer')(x)

model_2 = tf.keras.Model(inputs, outputs, name= 'model_2_LSTM')

In [127]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 LSTM_layer_1 (LSTM)         (None, 15, 64)            49408     
                                                                 
 global_avg_pool_layer (Glo  (None, 64)                0         
 balAveragePooling1D)                                            
                                                                 
 dense_layer (Dense)         (None, 64)               

In [128]:
model_2.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.legacy.Adam(),
                metrics= ['accuracy'])

In [129]:
model_2_history = model_2.fit(x = train_sentences,
                              y = train_labels,
                              epochs= 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVED_DIR, 'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240312-154939
Epoch 1/5
215/215 [==============================] - 7s 26ms/step - loss: 0.1251 - accuracy: 0.9545 - val_loss: 1.2042 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0629 - accuracy: 0.9714 - val_loss: 1.4600 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0465 - accuracy: 0.9775 - val_loss: 1.6172 - val_accuracy: 0.7520
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0409 - accuracy: 0.9778 - val_loss: 1.9617 - val_accuracy: 0.7480
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0424 - accuracy: 0.9764 - val_loss: 1.8052 - val_accuracy: 0.7559


In [130]:
model_2_pred = model_2.predict(val_sentences)
model_2_pred[:10]

24/24 [==============================] - 0s 6ms/step


array([[5.2121449e-02],
       [3.7462115e-01],
       [9.9999964e-01],
       [1.4338018e-01],
       [3.1619800e-07],
       [9.9999928e-01],
       [9.9982184e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9984944e-01]], dtype=float32)

In [131]:
model_2_results = evaluation(y_true= val_labels,
                             y_pred= tf.squeeze(tf.round(model_2_pred)))
model_2_results

{'accuracy': 0.7559055118110236,
 'precision': 0.7755102040816326,
 'recall': 0.6551724137931034,
 'f1-score': 0.7531121360163758}

In [132]:
np.array(list(model_2_results.values())) > np.array(list(baseline_results.values()))

array([False, False,  True, False])

### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit

In [140]:
# Build an RNN using the GRU cell
inputs = layers.Input(shape= (1, ), dtype= 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, name= 'GRU_layer_1')(x) # if you want to stack GRU cell return_sequences = True
# x = layers.LSTM(42, name= 'LSTM_layer_1', return_sequences= True)(x)
# x = layers.GRU(99, name= 'GRU_layer_2')(x)
# x = layers.Dense(64, activation= 'relu', name= 'dense_layer_1')(x)
# x = layers.GlobalAveragePooling1D(name= 'global_avg_pool_layer')(x)
outputs = layers.Dense(1, activation= 'sigmoid', name= 'output_layer')(x)
model_3 = tf.keras.Model(inputs, outputs, name= 'model_3_GRU')

In [141]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 GRU_layer_1 (GRU)           (None, 64)                37248     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [143]:
model_3.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.legacy.Adam(),
                metrics= ['accuracy'])

In [144]:
model_3_history = model_3.fit(x= train_sentences,
                                y= train_labels,
                                epochs= 5,
                                validation_data= (val_sentences, val_labels),
                                callbacks= [create_tensorboard_callback(SAVED_DIR, 'model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240312-161911
Epoch 1/5
215/215 [==============================] - 7s 23ms/step - loss: 0.1432 - accuracy: 0.9438 - val_loss: 0.8526 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0619 - accuracy: 0.9765 - val_loss: 0.9589 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0485 - accuracy: 0.9778 - val_loss: 1.1092 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0389 - accuracy: 0.9803 - val_loss: 1.3824 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0403 - accuracy: 0.9813 - val_loss: 1.5092 - val_accuracy: 0.7598


In [146]:
model_3_pred = model_3.predict(val_sentences)
model_3_pred

24/24 [==============================] - 0s 4ms/step


array([[1.45957386e-03],
       [7.60579944e-01],
       [9.99890208e-01],
       [1.72754258e-01],
       [1.05260398e-04],
       [9.99799073e-01],
       [9.97497857e-01],
       [9.99913931e-01],
       [9.99870777e-01],
       [9.99759734e-01],
       [4.68147569e-04],
       [9.99819338e-01],
       [6.29706716e-04],
       [1.40619487e-01],
       [1.07793210e-04],
       [1.38186326e-03],
       [1.95411063e-04],
       [2.74125428e-04],
       [7.56117776e-02],
       [9.99833584e-01],
       [9.99946713e-01],
       [1.16025723e-04],
       [9.99264777e-01],
       [3.48688685e-04],
       [9.99891877e-01],
       [9.99878645e-01],
       [3.58359597e-04],
       [4.92727559e-04],
       [7.12124573e-04],
       [9.93301868e-01],
       [9.99805987e-01],
       [3.42555752e-04],
       [5.10615390e-03],
       [1.87861151e-03],
       [9.59347904e-01],
       [3.84412438e-01],
       [9.99834538e-01],
       [1.98423192e-01],
       [9.77922510e-03],
       [9.99879837e-01],


In [150]:
model_3_results = evaluation(val_labels,
                             tf.squeeze(tf.round(model_3_pred)))
model_3_results

{'accuracy': 0.7598425196850394,
 'precision': 0.7522935779816514,
 'recall': 0.7068965517241379,
 'f1-score': 0.7590768821741806}

In [153]:
np.array(list(model_3_results.values())) > np.array(list(baseline_results.values()))

array([False, False,  True, False])

### Model 4: Bidirectional RNN

In [171]:
# Create bidirectional model
inputs = layers.Input(shape= (1, ), dtype= 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences= True), name= 'birectional_LSTM_layer_1')(x)
x = layers.Bidirectional(layers.GRU(64), name= 'bidirectional_LSTM_layer_2')(x)
# x = layers.GlobalAveragePooling1D(name= 'global_avg_pool_layer')(x)
outputs = layers.Dense(1, activation= 'sigmoid', name= 'output_layer')(x)

model_4 = tf.keras.Model(inputs, outputs, name= 'model_4_bidirectional')

In [172]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 birectional_LSTM_layer_1 (  (None, 15, 128)           98816     
 Bidirectional)                                                  
                                                                 
 bidirectional_LSTM_layer_2  (None, 128)               74496     
  (Bidirectional)                                                
                                             

In [173]:
model_4.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.legacy.Adam(),
                metrics= ['accuracy'])

In [174]:
model_4_history = model_4.fit(x= train_sentences,
            y= train_labels,
            epochs= 5,
            validation_data= (val_sentences, val_labels),
            callbacks= [create_tensorboard_callback(SAVED_DIR, 'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240312-191442
Epoch 1/5
215/215 [==============================] - 16s 51ms/step - loss: 0.0737 - accuracy: 0.9766 - val_loss: 1.2118 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 7s 33ms/step - loss: 0.0368 - accuracy: 0.9813 - val_loss: 1.6129 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 7s 32ms/step - loss: 0.0414 - accuracy: 0.9815 - val_loss: 1.6554 - val_accuracy: 0.7546
Epoch 4/5
215/215 [==============================] - 7s 30ms/step - loss: 0.0358 - accuracy: 0.9816 - val_loss: 1.4411 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 7s 31ms/step - loss: 0.0334 - accuracy: 0.9818 - val_loss: 1.7528 - val_accuracy: 0.7664


In [175]:
model_4_pred = model_4.predict(val_sentences)

24/24 [==============================] - 1s 21ms/step


In [176]:
model_4_results = evaluation(val_labels,
                             tf.squeeze(tf.round(model_4_pred)))
model_4_results

{'accuracy': 0.7664041994750657,
 'precision': 0.7814569536423841,
 'recall': 0.6781609195402298,
 'f1-score': 0.7642857598265804}

In [177]:
np.array(list(model_4_results.values())) > np.array(list(baseline_results.values()))

array([False, False,  True, False])

## CNN for Text and other types of sequences

We've used CNNs for image dataset which are 2D, however, our text data is 1D.

Previously for image we used Conv2D, but for text data we will use Conv1D

Architecture:
```
Input -> Tokenization -> Embedding -> CNN and Dense layers -> Output
```

### Model 5: 1D CNN

In [181]:
# Create the model
inputs = layers.Input(shape= (1, ), dtype= 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(64, 3, name= 'cnn_1d_layer_1')(x)
x = layers.MaxPool1D(name= 'max_pool_layer_1')(x)
x = layers.Conv1D(32, 3, name= 'cnn_1d_layer_2')(x)
x = layers.MaxPool1D(name= 'max_pool_layer_2')(x)
x = layers.GlobalAveragePooling1D(name= 'global_avg_pool_layer')(x)
outputs = layers.Dense(1, activation= 'sigmoid', name= 'output_layer')(x)

model_5 = tf.keras.Model(inputs, outputs, name= 'model_5_cnn_1d')

In [182]:
model_5.summary()

Model: "model_5_cnn_1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 cnn_1d_layer_1 (Conv1D)     (None, 13, 64)            24640     
                                                                 
 max_pool_layer_1 (MaxPooli  (None, 6, 64)             0         
 ng1D)                                                           
                                                                 
 cnn_1d_layer_2 (Conv1D)     (None, 4, 32)          

In [183]:
model_5.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.legacy.Adam(),
                metrics= ['accuracy'])

In [185]:
model_5_history = model_5.fit(x = train_sentences,
                              y= train_labels,
                              epochs= 5,
                              validation_data= (val_sentences, val_labels),
                              callbacks= [create_tensorboard_callback(SAVED_DIR, 'model_5_cnn_1d')])

Saving TensorBoard log files to: model_logs/model_5_cnn_1d/20240312-192748
Epoch 1/5
215/215 [==============================] - 7s 26ms/step - loss: 0.1346 - accuracy: 0.9526 - val_loss: 0.9021 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0730 - accuracy: 0.9742 - val_loss: 1.0374 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0604 - accuracy: 0.9766 - val_loss: 1.1580 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0584 - accuracy: 0.9752 - val_loss: 1.2200 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0532 - accuracy: 0.9787 - val_loss: 1.3146 - val_accuracy: 0.7533


In [186]:
model_5_pred = model_5.predict(val_sentences)

24/24 [==============================] - 0s 6ms/step


In [187]:
model_5_results = evaluation(val_labels,
                             tf.squeeze(tf.round(model_5_pred)))

In [188]:
model_5_results

{'accuracy': 0.7532808398950132,
 'precision': 0.7469135802469136,
 'recall': 0.6954022988505747,
 'f1-score': 0.7523500583553816}

In [189]:
np.array(list(model_5_results.values())) > np.array(list(baseline_results.values()))

array([False, False,  True, False])

### Model 6: Transfer learning

In [194]:
import tensorflow_hub as hub
embed = hub.load('https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/4')

In [197]:
embed_samples = embed(["When you complete with the downloading of the model then you will look into the encodings of this sentence"])
print(embed_samples[0][:50])

tf.Tensor(
[ 0.07687581  0.03288896  0.02179662 -0.04832654 -0.02299898 -0.00182735
 -0.03993054  0.08256393  0.01381563  0.03039743  0.01625713  0.02431568
  0.01514284  0.04843347  0.01244256  0.01343128  0.0023421  -0.02713761
 -0.00362572  0.00168669  0.01138663  0.0144208  -0.01719462 -0.03496176
 -0.07877127  0.04676209  0.03929747  0.02202034  0.01709515  0.0230419
  0.0915276  -0.03294362 -0.00317207 -0.04850543 -0.09554192  0.04421337
 -0.02980342  0.01039704 -0.0229306   0.06235658 -0.01796251 -0.02358888
 -0.02192312 -0.04135567  0.01966412  0.01150151 -0.0491185  -0.03221893
 -0.02000231 -0.04916013], shape=(50,), dtype=float32)


In [200]:
# Create Keras Layer using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2", 
                                        input_shape= [],
                                        dtype= 'string',
                                        trainable= False,
                                        name= 'use_layer')

In [213]:
# Create model using Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation='relu', name= 'dense_layer_1'),
    layers.Dense(1, activation= 'sigmoid', name= 'output_layer')
], name= 'model_6_USE')

model_6.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.legacy.Adam(),
                metrics= ['accuracy'])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 use_layer (KerasLayer)      (None, 512)               256797824 
                                                                 
 dense_layer_1 (Dense)       (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [214]:
model_6_history = model_6.fit(x= train_sentences,
            y= train_labels,
            epochs= 5,
            validation_data= (val_sentences, val_labels),
            callbacks= [create_tensorboard_callback(SAVED_DIR, 'model_6_USE')])

Saving TensorBoard log files to: model_logs/model_6_USE/20240312-232632
Epoch 1/5
215/215 [==============================] - 45s 196ms/step - loss: 0.4987 - accuracy: 0.7805 - val_loss: 0.4528 - val_accuracy: 0.8005
Epoch 2/5
215/215 [==============================] - 24s 114ms/step - loss: 0.4183 - accuracy: 0.8119 - val_loss: 0.4526 - val_accuracy: 0.8058
Epoch 3/5
215/215 [==============================] - 21s 97ms/step - loss: 0.4073 - accuracy: 0.8210 - val_loss: 0.4448 - val_accuracy: 0.8071
Epoch 4/5
215/215 [==============================] - 19s 89ms/step - loss: 0.4025 - accuracy: 0.8238 - val_loss: 0.4411 - val_accuracy: 0.8097
Epoch 5/5
215/215 [==============================] - 18s 85ms/step - loss: 0.3988 - accuracy: 0.8251 - val_loss: 0.4399 - val_accuracy: 0.8176


In [215]:
model_6_pred = model_6.predict(val_sentences)

24/24 [==============================] - 8s 311ms/step


In [216]:
model_6_results = evaluation(val_labels,
                             tf.squeeze(tf.round(model_6_pred)))
model_6_results

{'accuracy': 0.8175853018372703,
 'precision': 0.8317460317460318,
 'recall': 0.7528735632183908,
 'f1-score': 0.8165413215854364}

In [217]:
np.array(list(model_6_results.values())) > np.array(list(baseline_results.values()))

array([ True, False,  True,  True])

### Model 7: TF hub pretrained USE but with 10% of training data

Transfer learning helps alot when you don't have a large dataset.

Let's replicate `model_6` with only 10% of the training data

In [219]:
# Create subsets of 10% of the training data
train_10_percent = train_df_shuffled[['text', 'target']].sample(frac= 0.1, random_state= 42)
train_sentences_10_percent = train_10_percent['text'].to_list()
train_labels_10_percent = train_10_percent['target'].to_list()
len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [220]:
# Check the number of targets in our subsets of data
train_10_percent['target'].value_counts()

target
0    413
1    348
Name: count, dtype: int64

In [221]:
train_df_shuffled['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [222]:
X_train, X_test, y_train, y_test = train_test_split(train_sentences_10_percent, train_labels_10_percent, test_size= 0.1, random_state= 42)

In [223]:
model_7 = tf.keras.models.clone_model(model_6)
model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 use_layer (KerasLayer)      (None, 512)               256797824 
                                                                 
 dense_layer_1 (Dense)       (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [226]:
model_7.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.legacy.Adam(),
                metrics= ['accuracy'])

In [227]:
model_7_history = model_7.fit(X_train,
            y_train,
            epochs= 5,
            validation_data= (X_test, y_test),
            callbacks= [create_tensorboard_callback(SAVED_DIR, 'model_7_10_percent')])

Saving TensorBoard log files to: model_logs/model_7_10_percent/20240312-234523
Epoch 1/5
22/22 [==============================] - 14s 484ms/step - loss: 0.4176 - accuracy: 0.8129 - val_loss: 0.4413 - val_accuracy: 0.7922
Epoch 2/5
22/22 [==============================] - 5s 234ms/step - loss: 0.3969 - accuracy: 0.8319 - val_loss: 0.4420 - val_accuracy: 0.7922
Epoch 3/5
22/22 [==============================] - 4s 200ms/step - loss: 0.3817 - accuracy: 0.8421 - val_loss: 0.4385 - val_accuracy: 0.7792
Epoch 4/5
22/22 [==============================] - 4s 172ms/step - loss: 0.3690 - accuracy: 0.8450 - val_loss: 0.4410 - val_accuracy: 0.7792
Epoch 5/5
22/22 [==============================] - 4s 170ms/step - loss: 0.3601 - accuracy: 0.8523 - val_loss: 0.4395 - val_accuracy: 0.7662


In [229]:
model_7_pred = model_7.predict(X_test)

3/3 [==============================] - 2s 509ms/step


In [231]:
model_7_results = evaluation(y_test,
                             tf.squeeze(tf.round(model_7_pred)))
model_7_results

{'accuracy': 0.7662337662337663,
 'precision': 0.6756756756756757,
 'recall': 0.8064516129032258,
 'f1-score': 0.7683923463677226}

In [232]:
np.array(list(model_7_results)) > np.array(list(baseline_results))

array([False, False, False, False])